# Process NAMCS files for use

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from pandas.util.testing import assert_frame_equal
pd.set_option("display.max_columns",200)

In [2]:
# Pain 'diagnoses' according to the ICD-9-CM documentation followed by NAMCS
# Note that, as per NAMCS documentation guidelines, 
# any codes that don't have 2 decimal places have a '-' at the end
# There's an implied two-point decimal in each of these numbers
# from https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes.html

icd9 = pd.read_csv("data_files/icd9.csv")

In [3]:
# Drugs come from NCHS drug codes
# These can be found at https://www2.cdc.gov/drugs/applicationnav1.asp

# To strip whitespace from end of lines:
# find . -name "*.csv"  | xargs sed -i 's/[ \t]*$//'

drugs = pd.read_csv("data_files/drugs.csv", sep=',', dtype = {'drug':str,'code':str})

In [4]:
drugs.columns.tolist()

['drug', 'code', 'ingredient']

In [5]:
#append a '-' to 4-digit and '--' to 3-digit codes to make sure that all
# are in the NAMCS standard format
f = lambda x:  x + '-' if len(x) == 4 else x #check 4-digit nums
g = lambda x:  x + '--' if len(x) == 3 else x #check 3-digit nums

icd9["DIAGNOSIS CODE"] = icd9["DIAGNOSIS CODE"].map(f).map(g)

In [6]:
# This cell translates NAMCS2016's diagnosis column from ICD10 to ICD9 codes

In [7]:
# These are the fields to keep
fields = pd.read_csv("data_files/keep_cols.csv", names=['titles'])

In [8]:
# Thesse functions are for processing of data in certain years, as described in the cell below
def g(x):
    '''900000 means 'blank' in '93 and '94. In '94, 209900 and 209970 mean blank as well'''
    if str(x) not in ['900000', '209900', '209970', 'V9900','V9910','V9920','00000','V990-','V991-','V992-','V993-','V997-']:
        return x
    else:
        return np.nan 
def h(x):
    '''
    Only for '93 and '94
    if 1st digit is 1, remove it, if it starts with 2, then take away the 20 and put a 'V' in'''
    if len(str(x)) == 6:
        if str(x)[0] == '1':
            return str(x)[1:]
        else:
            return 'V' + str(x)[2:] 

In [9]:
# This loop adds all tables into a list, and drops all of the fields we don't care about as it goes
# This allows us to have a much smaller sized dataset to work with
year = []
# ACHTUNG!! Make sure you only add 2016 AFTER using the 2016 script to translate diagnoses to ICD-9 codes!
for i in range(1993,2017):
    tmp = pd.read_csv(f"NAMCS/namcs{str(i)}.csv",dtype=str)
    tmp.columns = tmp.columns.str.upper() # Make all column labels upper case
    cols = [col for col in tmp.columns.tolist() if col in list(fields['titles'])]
    tmp = tmp[cols]
    tmp = tmp.rename(index=str, columns={"ETHUN": "ETHNIC","ETHNICFL":"ETHNIC", "RACEUN": "RACE","PAYTYPER":"PAYTYPE","REGIONOFF":"REGION"}) #Make sure that we stack the correct columns on top of each other
    
    if i == 1993 or i == 1994:
        string = "DIAG1"
        tmp[string] =tmp[string].apply(g)
        tmp[string] =tmp[string].apply(h)

    year.append(tmp)
result = pd.concat(year, axis=0, join='outer', ignore_index=False, sort=False)

In [10]:
# In this cell we determine to keep the ETHUN column only if the ETHNIC column is null or nan
result['ETHNIC'] = result.apply(
    lambda row: row['ETHUN'] if pd.isna(row['ETHNIC']) else row['ETHNIC'],
    axis=1
)

In [11]:
# Add a columns called 'received' that is a 1 if the patient received an opiate, and 0 otherwise
# Note there are two medications allowed to every patient, but not 30

def opiate(df):
    if df['MED1'] in list(drugs['code']) or df['MED2'] in list(drugs['code']) or df['MED3'] in list(drugs['code']):
        return 1
    else:
        return 0

result['received'] = result.apply(opiate,axis=1)

In [12]:
# Replace all values that mean NaN with Nan
result  = result.replace({-9.0:np.nan,'-9.0':np.nan,-9:np.nan,'-9':np.nan,-7:np.nan,'-7':np.nan,-7.0:np.nan,'-7.0':np.nan,-8:np.nan,'-8':np.nan,-8.0:np.nan,'-8.0':np.nan})
result

,VMONTH,AGE,SEX,RACE,ETHNIC,RFV1,DIAG1,DIAG2,DIAG3,MED1,MED2,MED3,MED4,MED5,PATWT,REGION,YEAR,MED6,AGEDAYS,AGER,VDAYR,PREGNANT,PAYTYPE,MAJOR,TEMPF,BPSYS,BPDIAS,GLUCOSE,MED7,MED8,GESTWK,HTIN,WTLB,BMI,BONEDENS,ETHIM,RACEIM,MED9,MED10,DIAG4,DIAG5,SUBSTED,MED11,MED12,MED13,MED14,MED15,HTWTFL,received
0,10,6,1,1,1,14400,49390,159900,900000,60735,24355,33843,16475,09995,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,10,67,1,1,2,19201,72650,204430,115410,10126,05789,17165,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,10,47,1,1,2,41150,V6759,140190,127800,05789,19675,30553,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10,69,2,1,2,31000,18890,204589,900000,60735,17865,34495,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,10,42,2,1,2,10552,84890,900000,900000,19675,05810,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,10,46,1,1,2,11100,31100,172100,130510,01530,17365,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,10,4,1,1,2,13551,47390,146590,900000,01630,29705,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,10,2,2,1,2,13551,38290,146590,900000,01630,90000,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,10,58,1,1,2,22500,25000,900000,900000,15680,90000,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,10,60,1,1,2,19401,72610,117490,204589,19675,90000,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
result[result.received == 1]

,VMONTH,AGE,SEX,RACE,ETHNIC,RFV1,DIAG1,DIAG2,DIAG3,MED1,MED2,MED3,MED4,MED5,PATWT,REGION,YEAR,MED6,AGEDAYS,AGER,VDAYR,PREGNANT,PAYTYPE,MAJOR,TEMPF,BPSYS,BPDIAS,GLUCOSE,MED7,MED8,GESTWK,HTIN,WTLB,BMI,BONEDENS,ETHIM,RACEIM,MED9,MED10,DIAG4,DIAG5,SUBSTED,MED11,MED12,MED13,MED14,MED15,HTWTFL,received
88,8,21,2,2,2,19401,84890,900000,900000,32930,90000,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
89,8,41,1,4,2,10554,35010,900000,900000,32930,90000,90000,90000,90000,95686,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
153,3,49,1,1,2,26000,47390,153010,900000,08470,41780,90000,90000,90000,65651,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
198,8,18,1,1,2,32050,V2220,164663,900000,24975,32905,32920,90000,90000,26956,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
214,8,47,1,1,2,19401,72340,900000,900000,32920,90000,90000,90000,90000,26956,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
246,4,37,2,1,2,11000,30490,900000,900000,18985,90000,90000,90000,90000,27599,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
327,2,73,1,1,2,11000,40190,130000,173300,26045,30782,08470,17440,05789,77781,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
358,7,68,1,1,2,19201,45190,900000,900000,50035,19242,08470,90000,90000,58381,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
359,7,65,1,1,2,19251,71590,140190,199520,05347,08470,32905,90000,90000,58381,1,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
378,7,71,2,1,2,12201,71941,105319,900000,61100,34110,11152,90000,90000,89911,2,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [14]:
# Store the table as we currently have it, to prevent needing to remake it over and over
result.to_csv('data_files/result_original.csv')

In [15]:
# read in that pre-made table
result = pd.read_csv('data_files/result_original.csv')

/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,9,13,14,18,29,30,38,39,40,41,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
result[result.received == 1]

,Unnamed: 0,VMONTH,AGE,SEX,RACE,ETHNIC,RFV1,DIAG1,DIAG2,DIAG3,MED1,MED2,MED3,MED4,MED5,PATWT,REGION,YEAR,MED6,AGEDAYS,AGER,VDAYR,PREGNANT,PAYTYPE,MAJOR,TEMPF,BPSYS,BPDIAS,GLUCOSE,MED7,MED8,GESTWK,HTIN,WTLB,BMI,BONEDENS,ETHIM,RACEIM,MED9,MED10,DIAG4,DIAG5,SUBSTED,MED11,MED12,MED13,MED14,MED15,HTWTFL,received
88,88,8,21,2,2.0,2.0,19401.0,84890,900000,900000,32930.0,90000.0,90000.0,90000,90000,95686.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
89,89,8,41,1,4.0,2.0,10554.0,35010,900000,900000,32930.0,90000.0,90000.0,90000,90000,95686.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
153,153,3,49,1,1.0,2.0,26000.0,47390,153010,900000,8470.0,41780.0,90000.0,90000,90000,65651.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
198,198,8,18,1,1.0,2.0,32050.0,V2220,164663,900000,24975.0,32905.0,32920.0,90000,90000,26956.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
214,214,8,47,1,1.0,2.0,19401.0,72340,900000,900000,32920.0,90000.0,90000.0,90000,90000,26956.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
246,246,4,37,2,1.0,2.0,11000.0,30490,900000,900000,18985.0,90000.0,90000.0,90000,90000,27599.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
327,327,2,73,1,1.0,2.0,11000.0,40190,130000,173300,26045.0,30782.0,8470.0,17440,5789,77781.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
358,358,7,68,1,1.0,2.0,19201.0,45190,900000,900000,50035.0,19242.0,8470.0,90000,90000,58381.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
359,359,7,65,1,1.0,2.0,19251.0,71590,140190,199520,5347.0,8470.0,32905.0,90000,90000,58381.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
378,378,7,71,2,1.0,2.0,12201.0,71941,105319,900000,61100.0,34110.0,11152.0,90000,90000,89911.00000,2.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [17]:
opiates = result[result.received == 1]
opiates

,Unnamed: 0,VMONTH,AGE,SEX,RACE,ETHNIC,RFV1,DIAG1,DIAG2,DIAG3,MED1,MED2,MED3,MED4,MED5,PATWT,REGION,YEAR,MED6,AGEDAYS,AGER,VDAYR,PREGNANT,PAYTYPE,MAJOR,TEMPF,BPSYS,BPDIAS,GLUCOSE,MED7,MED8,GESTWK,HTIN,WTLB,BMI,BONEDENS,ETHIM,RACEIM,MED9,MED10,DIAG4,DIAG5,SUBSTED,MED11,MED12,MED13,MED14,MED15,HTWTFL,received
88,88,8,21,2,2.0,2.0,19401.0,84890,900000,900000,32930.0,90000.0,90000.0,90000,90000,95686.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
89,89,8,41,1,4.0,2.0,10554.0,35010,900000,900000,32930.0,90000.0,90000.0,90000,90000,95686.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
153,153,3,49,1,1.0,2.0,26000.0,47390,153010,900000,8470.0,41780.0,90000.0,90000,90000,65651.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
198,198,8,18,1,1.0,2.0,32050.0,V2220,164663,900000,24975.0,32905.0,32920.0,90000,90000,26956.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
214,214,8,47,1,1.0,2.0,19401.0,72340,900000,900000,32920.0,90000.0,90000.0,90000,90000,26956.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
246,246,4,37,2,1.0,2.0,11000.0,30490,900000,900000,18985.0,90000.0,90000.0,90000,90000,27599.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
327,327,2,73,1,1.0,2.0,11000.0,40190,130000,173300,26045.0,30782.0,8470.0,17440,5789,77781.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
358,358,7,68,1,1.0,2.0,19201.0,45190,900000,900000,50035.0,19242.0,8470.0,90000,90000,58381.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
359,359,7,65,1,1.0,2.0,19251.0,71590,140190,199520,5347.0,8470.0,32905.0,90000,90000,58381.00000,1.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
378,378,7,71,2,1.0,2.0,12201.0,71941,105319,900000,61100.0,34110.0,11152.0,90000,90000,89911.00000,2.0,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [18]:
# Store the df of patients who received opiates for later analysis
opiates.to_csv("data_files/opiate_receivers.csv")